In [16]:
output_dir = '/home/cloudlet/work/Hierarchical-Localization/outputs/argo_sample_2'
feature_path = Path(output_dir, 'feats-superpoint-n4096-r1024.h5')
assert feature_path.exists(), feature_path
feature_file = h5py.File(str(feature_path), 'r')
# print(feature_file.keys())
print(feature_file['ring_front_center_315978406032859416.jpg']['descriptors'].shape)
print(feature_file['ring_front_center_315978406032859416.jpg']['image_size'].shape)
print(feature_file['ring_front_center_315978406032859416.jpg']['keypoints'].shape)
print(feature_file['ring_front_center_315978406032859416.jpg']['scores'].shape)
print(feature_file['ring_front_center_315978406032859416.jpg']['keypoints'][666])

(256, 1207)
(2,)
(1207, 2)
(1207,)
[1462.9375  579.8125]


In [14]:
match_path = Path(output_dir, 'feats-superpoint-n4096-r1024_matches-superglue_pairs60_argo_query.h5')
assert match_path.exists(), match_path
match_file = h5py.File(str(match_path), 'r')
print(list(match_file.keys())[0])
print(match_file['ring_side_left_315978406032855224.jpg_ring_front_center_315978414557655080.jpg']['matches0'].shape)
print(match_file['ring_side_left_315978406032855224.jpg_ring_front_center_315978414557655080.jpg']['matching_scores0'].shape)

ring_side_left_315978406032855224.jpg_ring_front_center_315978414557655080.jpg
(1881,)
(1881,)


In [20]:
import numpy as np
import pickle
result_path = Path(output_dir, 'argo_hloc_superpoint+superglue_netvlad60.txt_logs.pkl')
with open(str(result_path), 'rb') as f:
    logs = pickle.load(f)
    

In [23]:
print(logs.keys())
query = list(logs['loc'].keys())[0]
print(query)

dict_keys(['features', 'matches', 'retrieval', 'loc'])
ring_side_left_315978415256957824.jpg


In [22]:
loc = logs['loc'][query]
print(loc.keys())

dict_keys(['db', 'PnP_ret', 'keypoints_query', 'points3D_xyz', 'points3D_ids', 'num_matches', 'keypoint_index_to_db'])


In [29]:
from hloc.utils.read_write_model import read_images_binary, read_points3d_binary
sfm_model = Path(output_dir, 'sfm_superpoint+superglue+netvlad/models/model_aligned')
images = read_images_binary(sfm_model / 'images.bin')
points3D = read_points3d_binary(sfm_model / 'points3D.bin')

In [33]:
print(len(list(images.keys())))
print(len(list(points3D.keys())))

785
110001


## Use NetVLAD to get the TOP 20 pairs for a query image
TODO: fix the cuda version conflicts between tf and pytorch
TODO: fix the environment conflicts between hfnet and hloc

In [35]:
import cv2
import numpy as np
import tensorflow as tf

import netvlad_tf.net_from_mat as nfm
import netvlad_tf.nets as nets

tf.reset_default_graph()

image_batch = tf.placeholder(
        dtype=tf.float32, shape=[None, None, None, 3])

net_out = nets.vgg16NetvladPca(image_batch)
saver = tf.train.Saver()

sess = tf.Session()
saver.restore(sess, nets.defaultCheckpoint())

inim = cv2.imread(nfm.exampleImgPath())
inim = cv2.cvtColor(inim, cv2.COLOR_BGR2RGB)
print(inim.shape)
batch = np.expand_dims(inim, axis=0)
print(batch.shape)
result = sess.run(net_out, feed_dict={image_batch: batch})
print(result.shape)

AttributeError: module 'tensorflow' has no attribute 'reset_default_graph'

In [ ]:
# DB images
import glob
from tqdm import tqdm
result_list = []
filename_list = []
dataset_path = '/home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2/'
for filename in tqdm(glob.glob(dataset_path + 'db/*.jpg')):
    filename_list.append(filename.split('/')[-1])
    inim = cv2.imread(filename)
    inim = cv2.cvtColor(inim, cv2.COLOR_BGR2RGB)
#     print(inim.shape)
    batch = np.expand_dims(inim, axis=0)
#     print(batch.shape)
    result = sess.run(net_out, feed_dict={image_batch: batch})
    result_list.append(result)
print(len(filename_list))
db = np.concatenate(result_list, axis=0)
print(db.shape)

In [ ]:
# Query image
base_dir = '/home/cloudlet/work/Hierarchical-Localization/test_query/'
result_list_query = []
filename_list_query = []
for filename in tqdm(glob.glob(base_dir + '*.jpg')):
    filename_list_query.append(filename.split('/')[-1])
    inim = cv2.imread(filename)
    inim = cv2.cvtColor(inim, cv2.COLOR_BGR2RGB)
#     print(inim.shape)
    batch = np.expand_dims(inim, axis=0)
#     print(batch.shape)
    result = sess.run(net_out, feed_dict={image_batch: batch})
    result_list_query.append(result)
print(filename_list_query)
print(len(filename_list_query))

In [ ]:
def compute_distance(desc1, desc2):
    # For normalized descriptors, computing the distance is a simple matrix multiplication.
    return 2 * (1 - desc1 @ desc2.T)

In [ ]:
pair_path = base_dir + 'pairs'
top_k = 20
import os
if not os.path.exists(pair_path):
    os.makedirs(pair_path)
with open(pair_path + 'pairs'+str(top_k)+'_argo_query.txt', 'w') as file:
    for i1, query in enumerate(result_list_query):
        query = query.flatten()
        dist = compute_distance(query, db)
        index_min_k = np.argsort(dist)[:top_k]

        for i2, db_file_index in enumerate(index_min_k):
            if i2 == len(index_min_k)-1 and i1 == len(result_list_query)-1:
                line = filename_list_query[i1] + ' ' + filename_list[db_file_index]
            else:
                line = filename_list_query[i1] + ' ' + filename_list[db_file_index] + '\n'
            file.write(line)

## Extract features

In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from hloc import extract_features, match_features, reconstruction, visualization
dataset = Path('/home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2')
images_db = dataset / 'db'
images_query = Path('test_query/query_image')

pairs = Path('test_query/pairs/')
loc_pairs = pairs / 'pairs20_argo_query.txt'  # top 20 retrieved by NetVLAD

outputs = Path('test_query/outputs/')
sfm_dir =  Path('outputs/argo_sample_2/sfm_superpoint+superglue+netvlad')

feature_conf = extract_features.confs['superpoint_aachen']
matcher_conf = match_features.confs['superglue']

features = feature_conf['output']
feature_file = f"{features}.h5"
match_file_query = f"{features}_{matcher_conf['output']}_{loc_pairs.stem}.h5"

In [45]:
extract_features.main(feature_conf, images_db, outputs/'db')

[10/28/2020 00:31:07 INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
Loaded SuperPoint model
[10/28/2020 00:31:07 INFO] Found 785 images in root /home/cloudlet/datasets/argoverse-tracking/sample/test_1017_2/db.


100%|██████████| 785/785 [00:12<00:00, 63.58it/s]

[10/28/2020 00:31:19 INFO] Finished exporting features.


In [46]:
extract_features.main(feature_conf, images_query, outputs/'query')

[10/28/2020 00:31:19 INFO] Extracting local features with configuration:
{'model': {'max_keypoints': 4096, 'name': 'superpoint', 'nms_radius': 3},
 'output': 'feats-superpoint-n4096-r1024',
 'preprocessing': {'grayscale': True, 'resize_max': 1024}}
Loaded SuperPoint model
[10/28/2020 00:31:19 INFO] Found 1 images in root test_query/query_image.


100%|██████████| 1/1 [00:00<00:00,  8.15it/s]

[10/28/2020 00:31:19 INFO] Finished exporting features.


# Match the query image with db images

In [49]:
import argparse
import torch
from pathlib import Path
import h5py
import logging
from tqdm import tqdm
import pprint

from hloc import matchers
from hloc.utils.base_model import dynamic_load
from hloc.utils.parsers import names_to_pair


'''
A set of standard configurations that can be directly selected from the command
line using their name. Each is a dictionary with the following entries:
    - output: the name of the match file that will be generated.
    - model: the model configuration, as passed to a feature matcher.
'''
confs = {
    'superglue': {
        'output': 'matches-superglue',
        'model': {
            'name': 'superglue',
            'weights': 'outdoor',
            'sinkhorn_iterations': 50,
        },
    },
    'NN': {
        'output': 'matches-NN-mutual-dist.7',
        'model': {
            'name': 'nearest_neighbor',
            'mutual_check': True,
            'distance_threshold': 0.7,
        },
    }
}


@torch.no_grad()
def matcher(conf, pairs, features, export_dir):
    logging.info('Matching local features with configuration:'
                 f'\n{pprint.pformat(conf)}')

    feature_db_path = Path(export_dir, 'db/'+features+'.h5')
    assert feature_db_path.exists(), feature_db_path
    feature_db_file = h5py.File(str(feature_db_path), 'r')

    feature_query_path = Path(export_dir, 'query/'+features+'.h5')
    assert feature_query_path.exists(), feature_query_path
    feature_query_file = h5py.File(str(feature_query_path), 'r')

    pairs_name = pairs.stem
    assert pairs.exists(), pairs
    with open(pairs, 'r') as f:
        pair_list = f.read().rstrip('\n').split('\n')

    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    print(device)
    Model = dynamic_load(matchers, conf['model']['name'])
    model = Model(conf['model']).eval().to(device)

    match_name = f'{features}_{conf["output"]}_{pairs_name}'
    match_path = Path(export_dir, match_name+'.h5')
    match_file = h5py.File(str(match_path), 'a')

    matched = set()
    for pair in tqdm(pair_list, smoothing=.1):
        name0, name1 = pair.split(' ')
        pair = names_to_pair(name0, name1)

        # Avoid to recompute duplicates to save time
        if len({(name0, name1), (name1, name0)} & matched) \
                or pair in match_file:
            continue

        data = {}
        feats0, feats1 = feature_query_file[name0], feature_db_file[name1]
        for k in feats1.keys():
            data[k+'0'] = feats0[k].__array__()
        for k in feats1.keys():
            data[k+'1'] = feats1[k].__array__()
        data = {k: torch.from_numpy(v)[None].float().to(device)
                for k, v in data.items()}

        # some matchers might expect an image but only use its size
        data['image0'] = torch.empty((1, 1,)+tuple(feats0['image_size'])[::-1])
        data['image1'] = torch.empty((1, 1,)+tuple(feats1['image_size'])[::-1])

        pred = model(data)
        grp = match_file.create_group(pair)
        matches = pred['matches0'][0].cpu().short().numpy()
        grp.create_dataset('matches0', data=matches)

        if 'matching_scores0' in pred:
            scores = pred['matching_scores0'][0].cpu().half().numpy()
            grp.create_dataset('matching_scores0', data=scores)

        matched |= {(name0, name1), (name1, name0)}

    match_file.close()
    logging.info('Finished exporting matches.')


In [50]:
matcher(matcher_conf, loc_pairs, features, outputs)

[10/28/2020 00:46:01 INFO] Matching local features with configuration:
{'model': {'name': 'superglue',
           'sinkhorn_iterations': 50,
           'weights': 'outdoor'},
 'output': 'matches-superglue'}
cuda


  5%|▌         | 1/20 [00:00<00:02,  9.08it/s]

Loaded SuperGlue model ("outdoor" weights)


100%|██████████| 20/20 [00:01<00:00, 15.79it/s]

[10/28/2020 00:46:03 INFO] Finished exporting matches.


## Find the 3D points given Bounding Box

In [2]:
from pathlib import Path
import h5py

In [3]:
feature_path = Path(outputs, 'db/feats-superpoint-n4096-r1024.h5')
assert feature_path.exists(), feature_path
feature_file = h5py.File(str(feature_path), 'r')
print(len(list(feature_file.keys())))
print(feature_file['ring_front_center_315978406032859416.jpg']['descriptors'].shape)
print(feature_file['ring_front_center_315978406032859416.jpg']['image_size'].shape)
print(feature_file['ring_front_center_315978406032859416.jpg']['keypoints'].shape)
print(feature_file['ring_front_center_315978406032859416.jpg']['scores'].shape)
# print(feature_file['ring_front_center_315978406032859416.jpg']['keypoints'][666])

785
(256, 1207)
(2,)
(1207, 2)
(1207,)


In [4]:
feature_path_q = Path(outputs, 'query/feats-superpoint-n4096-r1024.h5')
assert feature_path_q.exists(), feature_path_q
feature_file_q = h5py.File(str(feature_path_q), 'r')
print(len(list(feature_file_q.keys())))
print(feature_file_q['query.jpg']['descriptors'].shape)
print(feature_file_q['query.jpg']['image_size'].shape)
print(feature_file_q['query.jpg']['keypoints'].shape)
print(feature_file_q['query.jpg']['scores'].shape)
# print(feature_file['ring_front_center_315978406032859416.jpg']['keypoints'][666])

1
(256, 1508)
(2,)
(1508, 2)
(1508,)


In [5]:
match_path = Path(outputs, 'feats-superpoint-n4096-r1024_matches-superglue_pairs20_argo_query.h5')
assert match_path.exists(), match_path
match_file = h5py.File(str(match_path), 'r')
print(list(match_file.keys())[0])
print(match_file['query.jpg_ring_front_center_315978410361853528.jpg']['matches0'].shape)
print(match_file['query.jpg_ring_front_center_315978410361853528.jpg']['matching_scores0'].shape)

query.jpg_ring_front_center_315978410361853528.jpg
(1508,)
(1508,)


In [6]:
from hloc.utils.read_write_model import read_images_binary, read_points3d_binary
images = read_images_binary(sfm_dir / 'models/model_aligned/images.bin')
points3D = read_points3d_binary(sfm_dir / 'models/model_aligned/points3D.bin')
print(len(list(images.keys())))
print(len(list(points3D.keys())))

785
110001


In [25]:
import numpy as np

bbox = [80, 275, 55, 75] # [x,y,width,height]

def bbox_filter(bbox, query_keypoints):
    filtered_ids = []
    for i in range(query_keypoints.shape[0]):
        pos = query_keypoints[i]
        if pos[0] > bbox[0] and pos[0] < bbox[0]+bbox[2] and \
           pos[1] > bbox[1] and pos[1] < bbox[1]+bbox[3]:
            filtered_ids.append(i)
    return filtered_ids

filtered_ids = bbox_filter(bbox, feature_file_q['query.jpg']['keypoints'])
print(filtered_ids, '\n')

mean_pos_list = []
for match_name in list(match_file.keys()):
    db_img_name = match_name.split('jpg_')[-1]
    matches = match_file[match_name]['matches0']
    filtered_matches = [id for id in matches[filtered_ids] if id != -1]
    print(filtered_matches)
    
    db_img_id = list(feature_file.keys()).index(db_img_name)
    check_name = (images[db_img_id+1].name == db_img_name) # TODO: why need to + 1
    
    if check_name:
        keypoints = images[db_img_id+1].xys
        p3ids = images[db_img_id+1].point3D_ids
        print(keypoints.shape)
        print(p3ids.shape)
        selected_p3id3 = p3ids[filtered_matches]
        print(selected_p3id3)
        p3D = np.array([points3D[j].xyz for j in selected_p3id3 if j != -1])
        print(p3D, '\n')
        if p3D.size != 0:
            mean_pos = np.mean(p3D, axis = 0)
#             print(mean_pos, '\n')
            mean_pos_list.append(mean_pos)

mean_pos_arr = np.array(mean_pos_list)
print(mean_pos_arr)
def reject_outliers(data, m=2):
    return abs(data - np.mean(data)) < m * np.std(data)
mask = reject_outliers(mean_pos_arr[:, 2])
print(mask)
mean_pos_arr_filtered = mean_pos_arr[mask, :]
# print(mean_pos_arr_filtered)
print(np.mean(mean_pos_arr_filtered, axis=0))


[209, 230, 231, 232, 233, 262, 272, 291] 

[396, 446]
(1416, 2)
(1416,)
[81045 81133]
[[2616.50577184 1213.18699374   19.25434028]
 [2616.47963398 1213.33981656   18.85064648]] 

[448]
(1467, 2)
(1467,)
[81133]
[[2616.47963398 1213.33981656   18.85064648]] 

[445]
(1453, 2)
(1453,)
[81133]
[[2616.47963398 1213.33981656   18.85064648]] 

[]
(1490, 2)
(1490,)
[]
[] 

[386, 456]
(1503, 2)
(1503,)
[109578  81133]
[[2616.15898121 1213.6670485    19.25946615]
 [2616.47963398 1213.33981656   18.85064648]] 

[368, 413]
(1506, 2)
(1506,)
[ 80929 109420]
[[2616.19847176 1213.13001356   19.27126524]
 [2616.46106569 1213.31044779   18.91367291]] 

[335, 397]
(1507, 2)
(1507,)
[ 80929 109420]
[[2616.19847176 1213.13001356   19.27126524]
 [2616.46106569 1213.31044779   18.91367291]] 

[274, 269, 270, 314]
(1412, 2)
(1412,)
[ 80929  80930  80931 109420]
[[2616.19847176 1213.13001356   19.27126524]
 [2616.18767883 1213.37673094   19.24748593]
 [2616.13562578 1213.45941652   19.24777382]
 [2616.4610656